# 使用ID3、C4.5和CART方法建立决策树
实现：《机器学习》p93 习题4.3、4.4

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import operator

## 载入数据

In [2]:
with open('lenses.txt', 'r') as fr:  #加载文件
    lenses = [inst.strip().split('\t') for inst in fr.readlines()]  #处理文件，把文档中的字符串提取出来，保持行列关系
pd_data = pd.DataFrame(lenses, columns=['age', 'prescript', 'astigmatic', 'tearRate', 'category'])

In [3]:
pd_data

,age,prescript,astigmatic,tearRate,category
0,young,myope,no,reduced,no lenses
1,young,myope,no,normal,soft
2,young,myope,yes,reduced,no lenses
3,young,myope,yes,normal,hard
4,young,hyper,no,reduced,no lenses
5,young,hyper,no,normal,soft
6,young,hyper,yes,reduced,no lenses
7,young,hyper,yes,normal,hard
8,pre,myope,no,reduced,no lenses
9,pre,myope,no,normal,soft


In [4]:
from sklearn.model_selection import train_test_split
pd_train,pd_test = train_test_split(pd_data, test_size=0.2)

## 特征选择算法

In [5]:
#计算信息熵
def entropy(x, column='category'):
    prob = pd.value_counts(x[column]) / len(x[column])
    entropy = sum(np.log2(prob)*prob*(-1))
    return entropy

In [6]:
g_entropy = pd_train.groupby('age').apply(entropy) #计算每一个分组的信息熵
g_prop = pd.value_counts(pd_train['age'], normalize=True) #计算每一个分组的特征概率p
print(g_entropy,'\n', g_prop)

age
pre           1.378783
presbyopic    1.251629
young         1.459148
dtype: float64 
 pre           0.368421
young         0.315789
presbyopic    0.315789
Name: age, dtype: float64


In [7]:
entropy_base = entropy(pd_train)
c_entropy = 0
for each in g_entropy.keys():
    c_entropy += g_entropy[each] * g_prop[each] #条件熵
print('条件熵：{}'.format(c_entropy))
gain = entropy_base - c_entropy    #信息增益
print('信息增益：{}'.format(gain))

条件熵：1.3640077347838755
信息增益：0.019800000680207486


In [8]:
#计算条件熵的信息增益
def c_entropy_gain(x, column):
    g_entropy = x.groupby(column).apply(entropy) #计算每一个分组的信息熵
    g_prop = pd.value_counts(x[column], normalize= True) #计算每一个分组的特征概率p
    c_entropy = 0
    for each in g_entropy.keys():
        c_entropy += g_entropy[each] * g_prop[each] #条件熵
    entropy_base = entropy(x)          #信息熵
    gain = entropy_base - c_entropy    #信息增益
    return gain
#ID3返回值
def ID3(x):
    ID3_select = {}
    for i in x.columns[:-1]:
        ID3_select[i] = c_entropy_gain(x, i)
    #对字典的值进行排序,降序
    result = sorted(ID3_select.items(), key=operator.itemgetter(1), reverse=True)
    return result
#C4.5返回值
def C4_5(x):
    C4_5_select = {}
    for i in x.columns[:-1]:
        C4_5_select[i] = c_entropy_gain(x, i)/ entropy(x, i)
    #对字典的值进行排序,降序
    result = sorted(C4_5_select.items(), key=operator.itemgetter(1), reverse=True)
    return result

In [9]:
a = ID3(pd_train)
print(a)

[('tearRate', 0.7435552598651077), ('astigmatic', 0.40196447061915697), ('prescript', 0.1464511562889812), ('age', 0.019800000680207486)]


In [10]:
b = C4_5(pd_train)
print(b)

[('tearRate', 0.745044690722188), ('astigmatic', 0.42336470311763286), ('prescript', 0.15424808617541444), ('age', 0.012523474332019509)]


In [11]:
#gini index的计算
def gini(x, column='category'):
    p = pd.value_counts(x[column], normalize= True) #计算类别的特征概率p
    gini_index = 1 - (sum(p**2))
    return gini_index
#条件gini系数计算
def c_gini(x, column):
    g_gini = x.groupby(column).apply(gini) #计算每一个分组的gini
    g_prop = pd.value_counts(x[column], normalize= True) #计算每一个分组的特征概率p
    c_gini = 0
    for each in g_gini.keys():
        c_gini += g_gini[each] * g_prop[each] #条件gini
    return c_gini
#CART算法
def CART(x):
    CART_select = {}
    for i in x.columns[:-1]:
        CART_select[i] = c_gini(x, i)
    #对字典的值进行排序,升序
    result = sorted(CART_select.items(), key=operator.itemgetter(1), reverse=False)
    return result

In [12]:
c_gini(pd_train, 'age')

0.5614035087719298

In [13]:
CART(pd_train)

[('tearRate', 0.26900584795321636),
 ('astigmatic', 0.48746867167919794),
 ('prescript', 0.5401002506265664),
 ('age', 0.5614035087719298)]

## 建立决策树

In [14]:
pd_train.value_counts('category', sort=True).index[0]

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [ ]:
list(pd_train.groupby('age'))

In [ ]:
def tree(x, method, max_depth=-1, depth=0):
    '''
    Parameters:
        x: DataFrame数据
        method: ID3、C4_5、CART三种方法之一
        max_depth: 树最大深度,默认'-1'代表未设置
    Return:
        决策树字典
    '''
    if max_depth == 0:
        return x.value_counts('category', sort=True).index[0]
    elif isinstance(x, pd.core.frame.DataFrame):
        #使用gini系数判断“纯净”度
        if gini(x) == 0:
            return x.value_counts('category').index[0]  #纯净：返回类别名称
        else:
            column = method(x)[0][0]
            sub_dict = dict(list(x.groupby(column)))
            depth += 1
            for i in sub_dict.keys():
                _ = sub_dict[i].pop(column)
                if isinstance(sub_dict[i], pd.core.frame.DataFrame): #如果是DataFrame格式
                    if (depth < max_depth) or max_depth==-1:         #当深度不到最大深度时，递归调用
                        sub_dict[i] = tree(sub_dict[i], method, max_depth, depth)
                    else:                                            #否则赋值为出现频率最高的类别
                        sub_dict[i] = sub_dict[i].value_counts('category', sort=True).index[0]
            return {column: sub_dict}
    elif isinstance(x, str):
        pass

In [ ]:
#测试树准确率:单个样本
def test_one(tr, test):
    while isinstance(tr, dict):
        k = list(tr.keys())
        if k[0] in test.index:
            a = test[k[0]]
            tr = tr[k[0]][a]
        else:
            print('error')
    else:
        return tr
#样本集测试
def test_set(tr, test_set):
    '''
    Parameters:
        tr: 测试的决策树
        test_set: 测试集
    Return:
        准确率
    '''
    count = 0
    for i in range(len(test_set)):
        result = test_one(tr, test_set.iloc[i,:])
        if result == test_set.iloc[i,-1]:
            count += 1
    return count/len(test_set)

In [ ]:
ID3_tree = tree(pd_train, ID3)
print(ID3_tree)
print(test_set(ID3_tree, pd_test))

In [ ]:
#根据层预剪枝建立决策树，如果想实现按节点剪枝，建议使用自己创建的节点类建树。
def pre_pruning(x, test, method):
    best_tree = tree(x, method, max_depth=0)
    depth_now = 1
    while tree(x, method, max_depth=depth_now-1) !=  tree(x, method):
        if test_set(best_tree, test) < test_set(tree(x, method, max_depth=depth_now), test):
            best_tree = tree(x, method, max_depth=depth_now)
            depth_now += 1
        else:
            return best_tree
    else:
        return best_tree
pre_pruning(pd_train, pd_test, ID3)

In [ ]:
C4_5_tree = tree(pd_train, C4_5)
print(C4_5_tree)

In [ ]:
gini_tree = tree(pd_train, CART)
print(gini_tree)

## 画图

In [ ]:
from graphviz import Digraph


def plot_model(tree, name):
    g = Digraph("G", filename=name, format='png', strict=False)
    first_label = list(tree.keys())[0]
    g.node("0", first_label)
    _sub_plot(g, tree, "0")
    g.view()


root = "0"


def _sub_plot(g, tree, inc):
    global root

    first_label = list(tree.keys())[0]
    ts = tree[first_label]
    for i in ts.keys():
        if isinstance(tree[first_label][i], dict):
            root = str(int(root) + 1)
            g.node(root, list(tree[first_label][i].keys())[0])
            g.edge(inc, root, str(i))
            _sub_plot(g, tree[first_label][i], root)
        else:
            root = str(int(root) + 1)
            g.node(root, tree[first_label][i])
            g.edge(inc, root, str(i))

#图像和gv文件保存在tree目录下
plot_model(ID3_tree, "tree/ID3_tree.gv")
plot_model(C4_5_tree, "tree/C4_5_tree.gv")
plot_model(gini_tree, "tree/gini_tree.gv")

## 使用sklearn建立决策树

In [ ]:
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() #创建LabelEncoder()对象，用于序列化
for col in pd_train.columns[:-1]: #序列化
    pd_train[col] = le.fit_transform(pd_train[col])
    
clf = tree.DecisionTreeClassifier() #创建DecisionTreeClassifier()类
clf = clf.fit(pd_train.iloc[:,:-1].values.tolist(), pd_train.iloc[:,-1].values.tolist()) #使用训练数据（pd_train, tg_train），构建决策树

plt.figure(figsize=(30,30), dpi=30) #生成图片的像素大小=figsize * dpi → (900* 900)
tree.plot_tree(clf, feature_names= pd_train.iloc[:,:-1].keys(), class_names= clf.classes_, filled= True)
plt.savefig('tree/sklearn_tree.png')
plt.show()